In [ ]:
from apiclient import discovery
from httplib2 import Http
from oauth2client import client, file, tools
from google.oauth2 import service_account
from googleapiclient.discovery import build
import os

In [17]:
paths = ["./Profiles/Man/", "./Profiles/Woman/"]
MAN_PHOTO_URLS = []
MAN_TEXTS_URLS = []
WOMAN_PHOTO_URLS = []
WOMAN_TEXTS_URLS = []

PHOTO_URL_PATHS = ["Man_photo_urls.md", "Woman_photo_urls.md"]
TEXT_URL_PATHS = ["MAN_text_urls.md", "WOMAN_text_urls.md"]


# read the .md files and get the urls
for i, path in enumerate(PHOTO_URL_PATHS):
    with open(path, 'r') as file:
        for line in file:
            if i == 0:
                MAN_PHOTO_URLS.append(line)
            else:
                WOMAN_PHOTO_URLS.append(line)

for i, path in enumerate(TEXT_URL_PATHS):
    with open(path, 'r') as file:
        for line in file:
            if i == 0:
                MAN_TEXTS_URLS.append(line)
            else:
                WOMAN_TEXTS_URLS.append(line)

In [20]:
def get_question(img_url, text_url, index):
    question_updates = {
        "requests": [  
            # Adding an image item
            {
                "createItem": {
                    "item": {
                        "title": "Profile Picture",
                        "imageItem": {
                            "image": {
                                "sourceUri": img_url,  # URL of the image
                                "altText": ""
                            }
                        }
                    },
                    "location": {
                        "index": index
                    }
                }
            },
            {
                "createItem": {
                    "item": {
                        "title": "Profile Picture",
                        "imageItem": {
                            "image": {
                                "sourceUri": text_url,  # URL of the text
                                "altText": ""
                            }
                        }
                    },
                    "location": {
                        "index": index + 1
                    }
                }
            },
            # Adding a question item
            {
                "createItem": {
                    "item": {
                        "title": "Swipe Right?",
                        "questionItem": {
                            "question": {
                                "required": True,
                                "choiceQuestion": {
                                    "type": "RADIO",
                                    "options": [
                                        {"value": "Like :)"},
                                        {"value": "Dislike :("},
                                    ],
                                    "shuffle": False
                                }
                            }
                        }
                    },
                    "location": {
                        "index": index + 2
                    }
                }
            }
            
        ]
    }
    
    return question_updates

In [ ]:

# Replace with the correct path to your service account credentials file - replace example with actual name

SERVICE_ACCOUNT_FILE = "example.json"
SCOPES = ['https://www.googleapis.com/auth/forms.body', 'https://www.googleapis.com/auth/forms.responses.readonly']

# Authenticate with the service account credentials
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Build the Forms API client
service = build('forms', 'v1', credentials=creds)

# Example form creation request
form_title = {
    "info": {
        "title": "Dating Preferences!",
        "description": "Please fill out this form to help us find your perfect match."
    },
    # "update mask": "title, description"
}

requests = [
    {
        "createItem": {
            "item": {
                "title": "Tell us about your ideal partner. You can provide both a physical and a personality description.",
                "questionItem": {
                    "question": {
                        "textQuestion": {
                            "paragraph": True  # This sets the question as a paragraph text item
                        }
                    }
                }
            },
            "location": {
                "index": 0  # This adds the item at the start; adjust as needed
            }
        }
    }
]

service.forms().batchUpdate(formId=form_id, body=paragraph_update).execute()

In [ ]:
# q1 = get_question(MAN_PHOTO_URLS[0], MAN_TEXTS_URLS[0], 0)
# q2 = get_question(MAN_PHOTO_URLS[1], MAN_TEXTS_URLS[1], 3)

# Attempt to create the form
try:
    form = service.forms().create(body=form_title).execute()
    form_id = form['formId']
    print("Created Form URL:", form['responderUri'])
except Exception as e:
    print("Error creating form:", e)

# form_id = "example id"  

i = 0
for image_url, text_url in zip(MAN_PHOTO_URLS, MAN_TEXTS_URLS):
    print(image_url, text_url)
    # get the question
    question = get_question(image_url, text_url, i)
    print(question)
    service.forms().batchUpdate(formId=form_id, body=question).execute()
    i += 3
    # break

j = 0
for image_url, text_url in zip(WOMAN_PHOTO_URLS, WOMAN_TEXTS_URLS):
    print(image_url, text_url)
    # get the question
    question = get_question(image_url, text_url, j)
    service.forms().batchUpdate(formId=form_id, body=question).execute()
    print(question)
    j += 3

# try:
#     service.forms().batchUpdate(formId=form_id, body=q1).execute()
#     service.forms().batchUpdate(formId=form_id, body=q2).execute()
#     print("Questions added to the form.")
# except Exception as e:
#     print("Error updating form:", e)